In [2]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json
import pickle
import glob
import tqdm

pd.set_option("max.columns", 131)

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix
from scipy import sparse
import joblib as jb
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.read_csv("raw_data_all_labeled2.csv", index_col=0).dropna(subset=["y"])
df.shape

(1410, 15)

In [4]:
df.head()

,uploader,title,y,upload_date,user,view_count,like_count,dislike_count,thumbnail,width,height,categories,tags,channel_url,description
0,Yanjun Qi,S0-Introduction-Module3: Deep Learning and AI ...,0.0,2020-08-25,UCHMYETgeGbNHVHLidZSV8BQ,22,NaN,NaN,https://i.ytimg.com/vi/LkPmTGw1jqo/hqdefault.j...,1280,672,Science & Technology,Machine Learning,http://www.youtube.com/channel/UCHMYETgeGbNHVH...,Course Web: \nhttps://qiyanjun.github.io/2020f...
1,Ciência dos Dados,Machine Learning no Ensino Médio,0.0,2020-08-25,UCd3ThZLzVDDnKSZMsbK0icg,3,NaN,NaN,https://i.ytimg.com/vi_webp/R_gBq8IfwJc/maxres...,1920,1080,Education,machine learning|data science,http://www.youtube.com/channel/UCd3ThZLzVDDnKS...,"A matemática, sempre ela....\n\nDe uma maneira..."
2,iKennyHD,NBA LIVE 22: EA COULD USE DEEP MACHINE LEARNIN...,0.0,2020-08-25,KennyCallOfDuty,47,NaN,NaN,https://i.ytimg.com/vi/Tix2xon9MSs/maxresdefau...,1920,1080,Gaming,iKennyHD|nba live20|nba live 20|nba 2k20|live2...,http://www.youtube.com/channel/UCGMtoj9V9Go_im...,Wanna Donate? paypal.me/iKennyYT is where you ...
3,Amazon Web Services,Amazon Aurora Machine Learning – SageMaker Int...,0.0,2020-08-25,AmazonWebServices,335,NaN,NaN,https://i.ytimg.com/vi/w-2ip78NxAw/maxresdefau...,1920,1080,Science & Technology,AWS|Amazon Web Services|Cloud|AWS Cloud|Cloud ...,http://www.youtube.com/channel/UCd6MoB9NC6uYN2...,Learn how you can turn relational data into in...
4,"GMRIT, Rajam, AP",Machine Learning and Deep Learning Implementat...,1.0,2020-08-25,UC8g7hz4oXFzXNryt8h1gRPw,1486,NaN,NaN,https://i.ytimg.com/vi/f6XIY_M7FlA/hqdefault.j...,1280,720,People & Blogs,NaN,http://www.youtube.com/channel/UC8g7hz4oXFzXNr...,Resource Person\nMr.S.Aravinth Seshadri\nCerti...


In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [101]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']

## 1. Limpeza da data

In [102]:
df_limpo['date'] = pd.to_datetime(df['upload_date'])

## 2. Limpeza de Views

In [103]:
views = df['view_count'].fillna(0)
df_limpo['views'] = views

In [29]:
df_limpo.shape

(1410, 4)

## 3. Features

In [104]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()
today = datetime.datetime.today().strftime("%Y-%m-%d")
features['tempo_desde_pub'] = (pd.to_datetime(today) -  df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = (features['views'] / features['tempo_desde_pub']).round(3)
features.drop(['tempo_desde_pub'], axis=1, inplace=True)

In [105]:
features.shape

(1410, 2)

In [106]:
uploaders = df['uploader'].str.get_dummies()

In [108]:
df_limpo = pd.concat([df_limpo, uploaders], axis=1)

In [110]:
df_limpo.head()

,title,date,views,1littlecoder,365 Data Science,:CodeWebsDuh :,A.I.M Learning,AI Coding,AI Learner hub,AI era,AICamp,AIPRA,AP2V Academy,ASQStatsDivision,AWS Online Tech Talks,Aakash N S,Abacus AI,Abhishek Agarrwal,Abhishek Thakur,Abuzar Ghaffari,Academind,Accenture,Accenture in the Philippines,Acsia Technologies,Adriano Gianini,Adriano Marcos Rodrigues Figueiredo,Adventures in the Cloud,Affiliate World Conferences,Ahmed Nour,Akash Dash,Albert Coronado,Alberto Olla,Alex The Analyst,Alexander Amini,Algoritma Data Science School,Ali Nemati,All About GATE Exam,Alteryx,AltexSoft,Alura Cursos Online,AmandaLovesToAudit,Amanpour and Company,Amarpreet Singh,Amaze Lab,Amazon Web Services,Amod Sachintha,Analytics India Magazine,Analytics Vidhya,Andrew Schonfeld,Andrey Sozykin,André Furchner,Andy Jake,Aniedi Udo-Obong,Applied AI Course,Art of Engineer,Artificial Intelligence Tutorials,Arvind Kumawat,Ashutoshh Singh,Ashvin Nair,Association Quantum,Association for Computing Machinery (ACM),Asumsi,AtomsTalk,Australian Institute for Machine Learning,BEPEC - Career Transition Simplified,...,Whirldata,WhiteHatHacking,Wikitechy,XCTEQ Limited,Xuming Wang,YANGCOM Korea,Yanjun Qi,Yannic Kilcher,Yudi J,Yury Kashnitsky,Zach Star,ZaranTech,Zeeshan Usmani,a ydobon,akshay shekkari,ankitrathi.com,anıl Kaynar,ashish pondit,bespokeDS,biostatistique,codebasics,danny iskandar,datasciencearth,deeplizard,e-tube KTU by jasmin,eMaster Class Academy,eXtremegenerationIT,edX,edureka!,freeCodeCamp.org,geekbytes,iKennyHD,iNeuron iNtelligence,iT24Hrs,instituto cpfl,inzva team,jpmorgan,lakshay aggarwal,miracl6,nETSETOS,nejimakijima,njan,outcompete,pyGuru,sentdex,stanfordonline,stanley kan,study mart,suthichai live,teacher4u,techninja betechnical,upGrad,uΔΔTube,vcubingx,vectoroad,veryacademy,{‘フーテンのグラさん’:‘ジェントルふじふじ’},АйТиБорода,Продюсер будущего,Флесс,أكاديمية الفهرية Al-Fihriya Academy,へちやぼらけ・データサイエンティスト,エンジニアを目指すSESチャンネル,元専業Kaggler カレーちゃん,渡邉裕亮
0,S0-Introduction-Module3: Deep Learning and AI ...,2020-08-25,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Machine Learning no Ensino Médio,2020-08-25,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NBA LIVE 22: EA COULD USE DEEP MACHINE LEARNIN...,2020-08-25,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Amazon Aurora Machine Learning – SageMaker Int...,2020-08-25,335,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Machine Learning and Deep Learning Implementat...,2020-08-25,1486,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [111]:
for uploader in df_limpo.drop(['title', 'date', 'views'], axis=1).columns:
    features[uploader] = df_limpo[uploader]

In [112]:
mask_train = df_limpo['date'] < '2020-03-10'
mask_val = df_limpo['date'] >= '2020-03-10'

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((714, 642), (696, 642), (714,), (696,))

In [113]:
features[mask_train].shape

(714, 642)

In [114]:
mask_train = df_limpo['date'] < '2020-03-10'
mask_val = df_limpo['date'] >= '2020-03-10'

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((714, 642), (696, 642), (714,), (696,))

In [116]:
title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

# desc_train = df_limpo[mask_train]['description']
# desc_val = df_limpo[mask_val]['description']

# Min df - minimo de vezes que palavra tem que aparecer pra virar coluna
title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,2))

title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

# desc_bow_train = title_vec.fit_transform(desc_train)
# desc_bow_val = title_vec.transform(title_val)




In [117]:
title_bow_train.shape, desc_bow_train.shape

((714, 1134), (714, 17604))

In [118]:
# Concatenando as variávels numéricas com as geradas pelo TfidfVectorizer
Xtrain_wtitle = sparse.hstack([Xtrain, title_bow_train])
Xval_wtitle = sparse.hstack([Xval, title_bow_val])

In [119]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((714, 1776), (696, 1776))

# 4 RF

In [120]:
Xtrain_wtitle.shape

(714, 1776)

In [121]:
mdl_rf = RandomForestClassifier(n_estimators=1000, min_samples_leaf=1, random_state=0, class_weight='balanced', n_jobs=4)
mdl_rf.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=4,
                       random_state=0)

In [122]:
p_rf = mdl_rf.predict_proba(Xval_wtitle)[: ,1]

In [123]:
average_precision_score(yval, p_rf), roc_auc_score(yval, p_rf)

(0.40110169178238225, 0.6852483453580586)

In [76]:
#  (0.39545173300116343, 0.6776220857432473)


# 5 LGBM

In [124]:
params = [0.001862037165403463, 10, 20, 0.7594594688994479, 0.06663099351693305, 118, 3, 1]
lr = params[0]
max_depth = params[1]
min_child_samples = params[2]
subsample = params[3]
colsample_bytree = params[4]
n_estimators = params[5]

min_df = params[6]
ngram_range = (1, params[7])

title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = sparse.hstack([Xtrain, title_bow_train])
Xval_wtitle = sparse.hstack([Xval, title_bow_val])

mdl_lgbm = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                     min_child_samples=min_child_samples, subsample=subsample,
                     colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                     class_weight="balanced", n_jobs=6)
mdl_lgbm.fit(Xtrain_wtitle, ytrain)

p_lgbm = mdl_lgbm.predict_proba(Xval_wtitle)[:, 1]


/home/cesar/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [125]:
average_precision_score(yval, p_lgbm), roc_auc_score(yval, p_lgbm)

(0.3690408413294095, 0.6675392045793334)

# 7 Logistic Reg

In [126]:
from sklearn.pipeline import make_pipeline

In [127]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
#scaler = MaxAbsScaler()

#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())
#Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
#Xval_wtitle2 = scaler.transform(Xval_wtitle2)

lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(C=0.5, penalty='l2',n_jobs=6, random_state=0))
lr_pipeline.fit(Xtrain_wtitle2, ytrain)

Pipeline(steps=[('maxabsscaler', MaxAbsScaler()),
                ('logisticregression',
                 LogisticRegression(C=0.5, n_jobs=6, random_state=0))])

In [128]:
p_lr = lr_pipeline.predict_proba(Xval_wtitle2)[:, 1]

In [129]:
average_precision_score(yval, p_lr), roc_auc_score(yval, p_lr)

(0.39779914025803453, 0.6771569971975434)

# 8 Ensemble

(0.3969764593456903, 0.677681712479876) RF  
(0.4120975561037116, 0.677729413869179) LGBM  
(0.4176249062603176, 0.6702999224852424)LR  

(0.4050885250264036, 0.6983244887007335)LGBM NGRAM 2.3


In [130]:
p = (p_lr + p_rf + p_lgbm)/3
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.4175375271949755, 0.6966787907697811)

In [131]:
pd.DataFrame({"LR": p_lr, "RF": p_rf, "LGBM": p_lgbm}).corr()

,LR,RF,LGBM
LR,1.000000,0.735680,0.530397
RF,0.735680,1.000000,0.636368
LGBM,0.530397,0.636368,1.000000


In [133]:
p = 0.2*p_rf + 0.8*p_lgbm
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.4150782633651072, 0.6910082881163914)

(0.40302175815049546, 0.681360682129867) - 0.5/0.5  
(0.40535321271899116, 0.6836384234690835) - 0.4/0.6  
(0.40758321527756003, 0.686428954743307) - 0.3/0.7  
(0.414947642395506, 0.6905670502653388) - 0.2/0.8  

(0.4041888236276393, 0.6894460676167193) - 0.3/0.7 - lgbm ngram 1,3

(0.40306534693032414, 0.6860354182815573) - 0.4/0.6 

In [27]:
# reduzir complexidade do vectorizer

# 9 Salvar modelos

In [28]:
import joblib as jb

In [29]:
lgbm_20200911 = open('lgbm_20200911', 'wb')
pickle.dump(mdl_lgbm, lgbm_20200911)
lgbm_20200911.close()

In [30]:
random_forest_20200911 = open('random_forest_20200911', 'wb')
pickle.dump(mdl_rf, random_forest_20200911)
random_forest_20200911.close()

In [31]:
title_vectorizer_20200911 = open('title_vectorizer_20200911', 'wb')
pickle.dump(title_vec, title_vectorizer_20200911)
title_vectorizer_20200911.close()

In [32]:
jb.dump(mdl_lgbm, "lgbm_20200911.pkl.z")
jb.dump(mdl_rf, "random_forest_20200911.pkl.z")
#jb.dump(lr_pipeline, "logistic_reg_20200911.pkl.z")
jb.dump(title_vec, "title_vectorizer_20200911.pkl.z")

['title_vectorizer_20200911.pkl.z']

In [33]:
mdl_rf = pickle.load( open( "random_forest_20200911", "rb" ) )

In [34]:
from platform import python_version

print(python_version())

3.7.6


In [35]:
import sklearn

sklearn.__version__

'0.23.1'